## Import Libraries

In [1]:
import json

from prettytable import PrettyTable
from stringMatcher import levenshtein_ratio_and_distance
from time import time

table = PrettyTable(field_names=["Size", "Rating", "Reviews","Downloads", "Current App Version", "Device Requirements"])

## Load The Database

In [2]:
data = json.load(open("Datastore/dataFinal.json", "rb"))

## Load The Global Secondary Index

In [3]:
GIS = json.load(open("Datastore/dataFinal_GIS.json", "rb"))

## Write a function to search the movies by ```Title```

In [4]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the App_id which is the best match to the queryString...
    bestMatch_AppTitle = list(GIS)[bestMatchRatio_index]
    bestMatch_AppID    = GIS[bestMatch_AppTitle]

    # Now we have the App_id which is the primary search key in our main data-store...
    # Retrieve the details of the movie from the AppID
    appInfo            = data[str(bestMatch_AppID)]
    
    # Mark the search completion time...
    toc                  = time()

    # Add Application's Title to the response...
    appInfo["Title"] = bestMatch_AppTitle
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : appInfo
    }
    
    return responseModel

## Function to make the request and visualize the data

In [5]:
def request(appToSearch=""):
    # Application's name should always be in a string...
    appToSearch = str(appToSearch)
    # Check if the application name is valid or not...
    if(appToSearch.replace(" ", "").strip() == ""):
        return f"Invalid Movie Name"
    
    # Make a request to fetch the app info...
    response = getClosestMatch(queryString = appToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    # otherwise...
    # Construct the rating and specifications table...
    Size   = response["response body"]["Size"]
    Rating = response["response body"]["Average Rating"]
    Reviews = response["response body"]["Reviews"]
    Downloads = response["response body"]["Downloads"]
    Curr_Version = response["response body"]["Current Version"]
    Android = response["response body"]["Android Version"]
    
    #table
    table.add_row([Size, Rating, Reviews, Downloads, Curr_Version, Android])
    # Now we need to check if a user has given 
    data_to_print = f"\
    Title               : {response['response body']['Title']}\n\
    Genre               : {response['response body']['Genre']}\n\
    Ratings and Specs   : ---------------------------\n\
    "
    print(data_to_print)
    print(table)

## Time to Search

In [6]:
request(appToSearch="Watsap mesngar")

    Title               : WhatsApp Messenger
    Genre               : COMMUNICATION
    Ratings and Specs   : ---------------------------
    
+--------------------+--------+----------+----------------+---------------------+---------------------+
|        Size        | Rating | Reviews  |   Downloads    | Current App Version | Device Requirements |
+--------------------+--------+----------+----------------+---------------------+---------------------+
| Varies with device |  4.4   | 69109672 | 1,000,000,000+ |  Varies with device |  Varies with device |
+--------------------+--------+----------+----------------+---------------------+---------------------+


### Hence our code is complete and works even for mis-spelled queries